In [ ]:
from sklearn.neighbors import KernelDensity
import sys
sys.path.append('/mnt/users/hollowayp/paltas')
from KDE_one_stop import drop_extra_columns,hyperparam_range_dict,one_stop_kde
from Save_Summary_Batches import summary_batch
from squash_walkers import squash_walkers
from load_h5_file import load_h5_file
import matplotlib.pyplot as pl
from KDEpy import TreeKDE
import pandas as pd
import numpy as np
import pickle
import corner
from tqdm.notebook import tqdm
from sklearn.neighbors import KernelDensity

In [ ]:
with open('/mnt/extraspace/hollowayp/zBEAMS_data/class_instances/python3.11-Fiducial_0_0-63921.out_5_10_pickle.pkl','rb') as f:
    summary_batch_fiducial = pickle.load(f)

In [ ]:
from scipy.stats import distributions

def ks_weighted(data1, data2, wei1, wei2, alternative='two-sided'):
    ix1 = np.argsort(data1)
    ix2 = np.argsort(data2)
    data1 = data1[ix1]
    data2 = data2[ix2]
    wei1 = wei1[ix1]
    wei2 = wei2[ix2]
    data = np.concatenate([data1, data2])
    cwei1 = np.hstack([0, np.cumsum(wei1)/sum(wei1)])
    cwei2 = np.hstack([0, np.cumsum(wei2)/sum(wei2)])
    cdf1we = cwei1[np.searchsorted(data1, data, side='right')]
    cdf2we = cwei2[np.searchsorted(data2, data, side='right')]
    d = np.max(np.abs(cdf1we - cdf2we))
    # calculate p-value
    n1 = data1.shape[0]
    n2 = data2.shape[0]
    m, n = sorted([float(n1), float(n2)], reverse=True)
    en = m * n / (m + n)
    if alternative == 'two-sided':
        prob = distributions.kstwo.sf(d, np.round(en))
    else:
        z = np.sqrt(en) * d
        # Use Hodges' suggested approximation Eqn 5.3
        # Requires m to be the larger of (n1, n2)
        expt = -2 * z**2 - 2 * z * (m + 2*n)/np.sqrt(m*n*(m+n))/3.0
        prob = np.exp(expt)
    return d, prob



In [ ]:
d1 = np.random.normal(0,1,1000)
d2 = np.random.normal(0,1,1000)

In [ ]:
from scipy.stats import kstest

In [ ]:
JAX_chains_list_hyp = []
for chain_i in tqdm(range(len(summary_batch_fiducial.JAX_chains_list))):
    JAX_chains_list_hyp.append(squash_walkers(drop_extra_columns(summary_batch_fiducial.JAX_chains_list[chain_i].loc[:1999],
                                          Ok=True,alpha_weights_2=True)))

In [ ]:
kde_dict = {}
for chain in range(len(JAX_chains_list_hyp)):
    kde_dict[chain] = KernelDensity(bandwidth=0.01).fit(JAX_chains_list_hyp[chain]['OM'].to_numpy().reshape(-1, 1))

In [ ]:
hist_dict = {'bins':np.linspace(-0.1,1.1,121),'alpha':0.5,'density':True}
# pl.hist(kde_dict[0].sample(len(JAX_chains_list_hyp[0]['OM'])).flatten(),**hist_dict)
X_plot = np.linspace(-0.1,1.1,100)
pl.hist(JAX_chains_list_hyp[0]['OM'],**hist_dict)
pl.plot(X_plot,np.exp(kde_dict[0].score_samples(X_plot.reshape(-1,1))),c='k')
pl.show()

In [ ]:
def calculate_n_PCA(data):
    var_perc = 0.95
    pca_test = PCA(n_components=len(data.columns))  # Reduce to 10 dimensions (adjust based on data)
    pca_test_fit = pca_test.fit_transform(data)
    pca_variance = pca_test.explained_variance_ratio_/np.sum(pca_test.explained_variance_ratio_)
    n_PCA = np.arange(len(data.columns))[np.where(np.cumsum(pca_variance)>var_perc)[0]][0]
    print(f'Using {n_PCA} PCA components, which explain {var_perc}% of the variance')
    return n_PCA

In [ ]:
import copy

class KDE_scaling:
    def __init__(self,JAX_chains_list_hyp,N_dim,N_sys,N_iterations,bandwidth=0.05,shuffle_params=True,PCA_bool=False):
        np.random.seed(1)
        self.JAX_chains_list_hyp = copy.deepcopy(JAX_chains_list_hyp)
        self.bandwidth = bandwidth
        self.PCA_bool = PCA_bool
        del JAX_chains_list_hyp
        #Rescaling to std = 1:
        self.OM_std = np.mean([np.std(self.JAX_chains_list_hyp[elem]['OM']) for elem in range(len(self.JAX_chains_list_hyp))])
        self.std_array = np.mean([self.JAX_chains_list_hyp[elem].describe().loc['std'].to_numpy() for elem in range(len(self.JAX_chains_list_hyp))],axis=0)
        # print('Std',self.std_array)
        for b_i in range(len(self.JAX_chains_list_hyp)):
            self.JAX_chains_list_hyp[b_i]/=self.std_array
        # print('Describe',self.JAX_chains_list_hyp[0].describe())
        # print('Describe, unscaled',JAX_chains_list_hyp[0].describe())
        self.N_dim = N_dim
        #NOT including OM as this is included seperately later:
        self.param_list = ['Ode','w','wa']+\
                ['alpha_mu_0','alpha_mu_1','alpha_mu_2','alpha_weights_0','alpha_weights_1','alpha_scale_0','alpha_scale_1','alpha_scale_2']+\
                ['s_c','s_m','scale_c','scale_m']
        self.param_list_dict = {}
        self.N_sys = N_sys
        self.N_iterations = N_iterations
        self.kde_dict = {elem:[] for elem in range(len(self.JAX_chains_list_hyp))}
        self.indx_dict = {elem:[] for elem in range(len(self.JAX_chains_list_hyp))}
        self.JAX_chains_list_iterations = {elem:[] for elem in range(len(self.JAX_chains_list_hyp))}
        self.PCA_func = []
        self.PCA_transform = {elem:[] for elem in range(len(self.JAX_chains_list_hyp))}
        self.n_PCA_dict = {}
        for iteration_i in range(N_iterations):
            if shuffle_params: self.param_list_dict[iteration_i] = ['OM']+np.random.choice(self.param_list,N_dim-1,replace=False).tolist()
            else: self.param_list_dict[iteration_i] = ['OM']+self.param_list[:N_dim-1]
            self.n_PCA_dict[iteration_i] = calculate_n_PCA(self.JAX_chains_list_hyp[0][self.param_list_dict[iteration_i]])
            for chain in range(len(self.JAX_chains_list_hyp)):
                random_indx = np.random.choice(np.arange(len(self.JAX_chains_list_hyp[chain])),N_sys,replace=False)
                self.indx_dict[chain].append(random_indx)
                self.JAX_chains_list_iterations[chain].append(self.JAX_chains_list_hyp[chain].loc[random_indx])
                if PCA_bool:
                    if chain==0: self.PCA_func.append(PCA(n_components=self.n_PCA_dict[iteration_i],whiten=True))
                    if chain==0: self.PCA_transform[chain].append(self.PCA_func[iteration_i].fit_transform(self.JAX_chains_list_iterations[chain][iteration_i][self.param_list_dict[iteration_i]].to_numpy()))
                    else: self.PCA_transform[chain].append(self.PCA_func[iteration_i].transform(self.JAX_chains_list_iterations[chain][iteration_i][self.param_list_dict[iteration_i]].to_numpy()))
        for iteration_i in range(N_iterations):
            for chain in range(len(self.JAX_chains_list_hyp)):
                if PCA_bool: 
                    kde_input_data = self.PCA_transform[chain][iteration_i]
                    if self.n_PCA_dict[iteration_i]==1: kde_input_data = kde_input_data.reshape(-1,1)
                    # print('KDE input',kde_input_data.shape,np.std(kde_input_data,axis=0))
                else: 
                    kde_input_data = self.JAX_chains_list_iterations[chain][iteration_i][self.param_list_dict[iteration_i]].to_numpy()
                    if self.N_dim==1: kde_input_data = kde_input_data.reshape(-1,1)
                self.kde_dict[chain].append(KernelDensity(bandwidth=bandwidth).fit(kde_input_data))
        # print('Param list:',self.param_list_dict)
    def find_product(self):
        assert len(self.JAX_chains_list_hyp)==2 #Am only finding the product with the 2nd index.
        self.prod_dict = {elem:[] for elem in range(len(self.JAX_chains_list_hyp))}
        for iteration_i in range(self.N_iterations):
            for chain in range(len(self.JAX_chains_list_hyp)):
                if self.PCA_bool:
                    if self.N_dim==1: prod_i = np.exp(self.kde_dict[chain][iteration_i].score_samples(self.PCA_func[iteration_i].transform(self.JAX_chains_list_hyp[1-chain][self.param_list_dict[iteration_i]].to_numpy()).reshape(-1,1)))
                    else: prod_i = np.exp(self.kde_dict[chain][iteration_i].score_samples(self.PCA_func[iteration_i].transform(self.JAX_chains_list_hyp[1-chain][self.param_list_dict[iteration_i]].to_numpy())))
                    # print('Prod shape',prod_i.shape)
                else:
                    if self.N_dim==1: prod_i = np.exp(self.kde_dict[chain][iteration_i].score_samples(self.JAX_chains_list_hyp[1-chain][self.param_list_dict[iteration_i]].to_numpy().reshape(-1,1)))
                    else: prod_i = np.exp(self.kde_dict[chain][iteration_i].score_samples(self.JAX_chains_list_hyp[1-chain][self.param_list_dict[iteration_i]].to_numpy()))
                self.prod_dict[chain].append(prod_i)
        return self
    def compare_1D_histogram(self,plot=False):
        try: self.prod_dict
        except: self.find_product()
        hist_dict_2 = {'bins':np.linspace(0,1,51),'density':True}
        plot_hist_dict_2 = {'bins':np.linspace(-0.1,1.1,51),'density':True,'fill':False,'linewidth':2}
        ratio_list = []
        for iteration_i in range(self.N_iterations):
            for chain_i in range(1):#range(len(self.JAX_chains_list_hyp)): #Degenerate with 1-chain.
                #Applying the std back to the OM:
                w1 = self.prod_dict[1-chain_i][iteration_i]
                w2 = self.prod_dict[chain_i][iteration_i]
                # print('weights',self.JAX_chains_list_hyp[chain_i]['OM'].shape,w1.shape)
                f1,b1 = np.histogram(self.JAX_chains_list_hyp[chain_i]['OM']*self.OM_std,weights=w1,**hist_dict_2)
                f2,b2 = np.histogram(self.JAX_chains_list_hyp[1-chain_i]['OM']*self.OM_std,weights=w2,**hist_dict_2)
                ratio_list.append(np.nanmedian(abs(f1-f2)/f2))
                if plot:
                    print('Iteration',iteration_i,'Params',self.param_list_dict[iteration_i])
                    if self.PCA_bool: fig,ax = pl.subplots(figsize=(7,5));ax_0=ax
                    else:fig,ax = pl.subplots(1,2,figsize=(10,5));ax_0=ax[0];ax_1=ax[1]
                    ax_0.hist(self.JAX_chains_list_hyp[chain_i]['OM']*self.OM_std,weights=self.prod_dict[1-chain_i][iteration_i],
                            **plot_hist_dict_2,edgecolor='darkorange')
                    ax_0.hist(self.JAX_chains_list_hyp[1-chain_i]['OM']*self.OM_std,weights=self.prod_dict[chain_i][iteration_i],
                            **plot_hist_dict_2,edgecolor='darkblue')
                    ax_0.set_xlabel('OM')
                    ax_0.set_title(f'Product of KDEs in {self.N_dim}D: Bandwidth: {self.bandwidth}')
                    KDE_1D = KernelDensity(bandwidth=self.bandwidth).fit(self.JAX_chains_list_hyp[chain]['OM'].loc[self.indx_dict[chain][iteration_i]].to_numpy().reshape(-1,1))
                    if not self.PCA_bool:
                        ax_1.hist(self.JAX_chains_list_hyp[chain_i]['OM']*self.OM_std,
                            **plot_hist_dict_2,edgecolor='grey',alpha=0.5)
                        ax_1.hist(self.JAX_chains_list_hyp[1-chain_i]['OM']*self.OM_std,
                            **plot_hist_dict_2,edgecolor='grey',alpha=0.5)
                        ax_1.set_xlabel('OM')
                        ax_1.set_title(f'KDE in 1D: Bandwidth: {self.bandwidth}')
                        X_plot = np.linspace(-0.1,1.1,100)
                        ax_1.plot(X_plot,np.exp(KDE_1D.score_samples(X_plot.reshape(-1,1)/self.OM_std))/self.OM_std,c='k')                    
                    if plot: pl.show()
        self.ratio_list = ratio_list
        return self

mean_ratio = {}
mean_ratio_std = {}
N_sys_list = [1000,5000,10000,15000]
N_dim_list = np.arange(3,8)
N_dim_max = np.max(N_dim_list)
N_iterations=3
bandwidth_list = [0.01,0.05,0.1,0.2]
for bandwidth in bandwidth_list:
    for N_dim in N_dim_list:
        mean_ratio[(N_dim,bandwidth)] = []
        mean_ratio_std[(N_dim,bandwidth)] = []
        for N_sys in tqdm(N_sys_list):
            ratio_list_i = KDE_scaling(JAX_chains_list_hyp,N_dim=N_dim,
                                       N_sys=N_sys,N_iterations=N_iterations,
                                       bandwidth=bandwidth,PCA_bool=True).compare_1D_histogram(plot=False).ratio_list
            print(f'Ratio list: {ratio_list_i}')
            mean_ratio[(N_dim,bandwidth)].append(np.nanmean(ratio_list_i))
            mean_ratio_std[(N_dim,bandwidth)].append(np.nanstd(ratio_list_i))
            plot_scaling_relation(N_dim,N_sys_list,mean_ratio,mean_ratio_std,
                                  bandwidth_list=bandwidth_list,N_dim_max=N_dim_max,
                                saveas='/mnt/zfsusers/hollowayp/zBEAMS/KDE_scaling_relation_plot_test'
                                )

In [ ]:
fig,ax = pl.subplots(1,len(JAX_chains_list_hyp),figsize=(5*len(JAX_chains_list_hyp),5))
for b_i in range(len(JAX_chains_list_hyp)):
    ax[b_i].hist(JAX_chains_list_hyp[b_i]['OM'],bins=np.linspace(0,1,51),density=True,alpha=0.5)
pl.show()


In [ ]:
large_dataset = [pd.concat([JAX_chains_list_hyp[0],JAX_chains_list_hyp[1]]).reset_index(drop=True),
                pd.concat([JAX_chains_list_hyp[2],JAX_chains_list_hyp[3]]).reset_index(drop=True)]
KDE_large = KDE_scaling(large_dataset,N_dim=16,
            N_sys=len(large_dataset[0]),N_iterations=1,
            bandwidth=0.2,PCA_bool=True).compare_1D_histogram(plot=True)

In [ ]:
KDE_scaling(JAX_chains_list_hyp,N_dim=N_dim,
                    N_sys=N_sys,N_iterations=N_iterations,bandwidth=bandwidth).compare_1D_histogram(plot=(N_dim>4)).ratio_list


In [ ]:
plot_scaling_relation(N_dim,N_sys_list,mean_ratio,mean_ratio_std,
                                  bandwidth_list=bandwidth_list,N_dim_max=N_dim_max,
                                saveas='/mnt/zfsusers/hollowayp/zBEAMS/test_figure_can_delete'
                                )

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=16)  # Reduce to 10 dimensions (adjust based on data)
X_f_reduced = pca.fit_transform(JAX_chains_list_hyp[0])
print(np.arange(16)[np.where(np.cumsum(pca.explained_variance_ratio_/np.sum(pca.explained_variance_ratio_))>0.99)[0]])
print(np.cumsum(pca.explained_variance_/np.sum(pca.explained_variance_)))
pl.plot(pca.explained_variance_/np.sum(pca.explained_variance_))
pl.show()

In [ ]:
def plot_scaling_relation(N_dim,N_sys_list,mean_ratio,mean_ratio_std,bandwidth_list,N_dim_max=3,saveas=None):
    fig = pl.figure(figsize=(7,5))
    pl.title('KDE Scaling',size=18)
    linestyle_list = ['-','--',':','-.','--',':','-.']
    color_dict = {elem+1:pl.cm.viridis(np.linspace(0,1,N_dim_max))[elem] for elem in range(N_dim_max)}
    for p_i,key_i in enumerate(mean_ratio_std.keys()):
        N_dim,bandwidth = key_i
        linestyle_i = linestyle_list[np.where(np.array(bandwidth_list)==bandwidth)[0][0]]
        pl.errorbar(N_sys_list[:len(mean_ratio[(N_dim,bandwidth)])],mean_ratio[(N_dim,bandwidth)],
                        yerr=mean_ratio_std[(N_dim,bandwidth)],
                        linestyle=linestyle_i,
                    c=color_dict[N_dim],label=f'{N_dim}D,B:{bandwidth}')
    pl.xlabel('Number of systems',size=15)
    pl.ylabel('Median bin ratio',size=15)
    fig.legend(fontsize=7,bbox_to_anchor=(1.12,0.5),loc='center right')
    pl.ylim(-0.1,2.1)
    pl.tight_layout()
    if saveas is not None:
        for fmt in ['pdf','png']: pl.savefig(saveas+f'.{fmt}',dpi=500,bbox_inches='tight')
    pl.close()

In [ ]:
X_plot = np.linspace(-2,2,100)
Y_1D = np.exp(KernelDensity().fit([[0]]).score_samples(X_plot.reshape(-1,1)))
Y_2D = np.exp(KernelDensity().fit([[0,0]]).score_samples(np.array([X_plot,np.zeros(len(X_plot))]).T))

pl.plot(X_plot,Y_1D/np.max(Y_1D),label='1D')
pl.plot(X_plot,Y_2D/np.max(Y_2D),'--',label='2D')
pl.ylim(bottom=0)
pl.legend()
pl.show()